In [14]:
import csv
import sqlite3
import csv
SQLLITEFILE = '/home/jay/BigData/SFUCourses/CMPT733/Project/Data/latlongadd_db.sqlite'

sqlite_file = SQLLITEFILE    # name of the sqlite database file

# Connecting to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

VANCOUVER_MERGED_FILE = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/businesswise_vancouver.csv/part-00000-cd0a3266-fea3-4970-bea0-25df4c55e277.csv"
VANCOUVER_FINAL = "/home/jay/BigData/SFUCourses/CMPT733/Project/Data/vancouver_final.csv"
def format_string(val):
    if val == "":
        val = ""
    else:
        val = val.replace("'","''")
    return val

def find_latlong(house, street, city, province, country, postalcode):
    latitude=0
    longitude=0
    
    sql_text = """SELECT Latitude, Longitude 
    FROM AddressLatLong 
    WHERE House='{0}' 
    AND Street = '{1}'
    AND City ='{2}'
    AND Province = '{3}'
    AND Country = '{4}'
    AND PostalCode = '{5}' LIMIT 1""".format(format_string(house), format_string(street),
                 format_string(city), format_string(province), 
                 format_string(country), format_string(postalcode))
    #print sql_text
    c.execute(sql_text)
    csvline = c.fetchall()
    if len(csvline) > 0:
        latitude, longitude = csvline[0]
    #print latitude, longitude
    return latitude, longitude
        

def replace_latitude_longitude(filelocation, outputfile):
    f=open(outputfile, "w" )
    
    with open(filelocation, 'r') as filepointer:
        csvreader = csv.reader(filepointer)
        csvreader.next()
        for csvline in csvreader:
            house = csvline[6]
            street = csvline[7]
            city =csvline[8]
            province = csvline[9]
            country = csvline[10]
            postalcode = csvline[11]
            latitude = csvline[12]
            longitude = csvline[13]
            if latitude == "" or longitude == "":
                latitude, longitude = find_latlong(house, street, city, province, country, postalcode)
                if latitude != 0 or longitude != 0:
                    csvline[12] = str(latitude)
                    csvline[13] = str(longitude)
            f.write(','.join(csvline)+"\n")
            #print csvline
            #break
                
replace_latitude_longitude(VANCOUVER_MERGED_FILE, VANCOUVER_FINAL)    
    